### Importar Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import optuna 
import shap
import seaborn as sns

from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve, precision_score, recall_score, f1_score, precision_recall_fscore_support
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

import joblib

sns.set(style="whitegrid")
%matplotlib inline

## Carregar Dados

In [2]:
df = pd.read_csv('../data/processed/employee_attrition_processed.csv')

df.head()

,Age,Attrition,DailyRate,DistanceFromHome,Education,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,...,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes
0,41,1,1102,1,2,1,2,0,94,3,...,False,False,False,False,False,True,False,False,True,True
1,49,0,279,8,1,2,3,1,61,2,...,False,False,False,False,True,False,False,True,False,False
2,37,1,1373,2,2,4,4,1,92,2,...,True,False,False,False,False,False,False,False,True,True
3,33,0,1392,3,4,5,4,0,56,3,...,False,False,False,False,True,False,False,True,False,True
4,27,0,591,2,1,7,1,1,40,3,...,True,False,False,False,False,False,False,True,False,False


tudo certo!

- Attrition virou 0/1

- Gender virou 0/1

- OverTime, MaritalStatus, JobRole, etc., viraram dummies (True/False)

### train_test_split

In [3]:
X = df.drop('Attrition', axis=1)
y = df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

print(f"Tamanho do treino: {X_train.shape}")
print(f"Tamanho do teste: {X_test.shape}")

Tamanho do treino: (1176, 47)
Tamanho do teste: (294, 47)


### Smote

In [4]:
smote = SMOTE(random_state=42)
    
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    
print("Distribuição original:", y_train.value_counts(normalize=True))
print("Distribuição após SMOTE:", y_train_resampled.value_counts(normalize=True))

Distribuição original: Attrition
0    0.838435
1    0.161565
Name: proportion, dtype: float64
Distribuição após SMOTE: Attrition
0    0.5
1    0.5
Name: proportion, dtype: float64


### K-fold

In [5]:
neg, pos = Counter(y_train_resampled)[0], Counter(y_train_resampled)[1]
spw = neg / pos

xgb_kfold = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=spw,
    **study.best_params
)

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(
    xgb_kfold,
    X_train_resampled, 
    y_train_resampled,
    scoring='f1',
    cv=cv
)

print(f"F1 CV média: {scores.mean():.4f} ± {scores.std():.4f}")

NameError: name 'study' is not defined

modelo está performando muito bem no conjunto de treino balanceado.

### XGBoost

In [ ]:
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb_model.fit(X_train_resampled, y_train_resampled)

NameError: name 'X_train_resampled' is not defined

In [ ]:
y_pred_xgb = xgb_model.predict(X_test)
print("=== XGBoost ===")
print(confusion_matrix(y_test, y_pred_xgb))
print(classification_report(y_test, y_pred_xgb))
print("ROC AUC XGB:", roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:, 1]))

NotFittedError: need to call fit or load_model beforehand

XGBoost

- Acurácia: 0.87

- ROC AUC: 0.78

- F1-Score: 0.47 para a classe 1

### Optuna

In [ ]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 5),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 5)
    }

    model = XGBClassifier(
        use_label_encoder=False,
        eval_metric='logloss',
        random_state=42,
        **params
    )

    model.fit(X_train_resampled, y_train_resampled)

    y_pred = model.predict(X_test)

    return f1_score(y_test, y_pred, pos_label=1)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, timeout=600) 


NameError: name 'optuna' is not defined

In [ ]:
print("Melhores parâmetros:")
print(study.best_params)

print("Melhor F1-score para classe 1:")
print(study.best_value)

Melhores parâmetros:


NameError: name 'study' is not defined

Com esse contexto, um F1-score de 0.5714 indica que o modelo está razoavelmente equilibrado entre a precisão e o recall, mas ainda não é ótimo. Isso é um indicativo de que o modelo consegue identificar a demissão de um funcionário (classe 1) com alguma precisão, mas não de forma robusta o suficiente para ser considerado confiável em um cenário crítico.

### Melhorar F1-score

In [ ]:
best_params = study.best_params

neg, pos = Counter(y_train_resampled)[0], Counter(y_train_resampled)[1]
spw = neg / pos

xgb_final = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42,
    scale_pos_weight=spw,
    **best_params
)

xgb_final.fit(X_train_resampled, y_train_resampled)

probs = xgb_final.predict_proba(X_test)[:, 1]

thresholds = np.arange(0.1, 0.9, 0.01)

results = []
for t in thresholds:
    preds = (probs >= t).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, preds, average='binary')
    results.append((t, precision, recall, f1))

th_df = pd.DataFrame(results, columns=['Threshold','Precision','Recall','F1'])

best = th_df.loc[th_df['F1'].idxmax()]
print("🎯 Melhor Threshold encontrado:")
print(best)


NameError: name 'study' is not defined

In [ ]:
neg, pos = Counter(y_train_resampled)[0], Counter(y_train_resampled)[1]
spw = neg / pos

xgb_final = XGBClassifier(
    use_label_encoder=False, eval_metric='logloss', random_state=42,
    scale_pos_weight=spw,
    **study.best_params
)
xgb_final.fit(X_train_resampled, y_train_resampled)

from sklearn.model_selection import StratifiedKFold, cross_val_score

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(
    xgb_final,
    X_train_resampled, y_train_resampled,
    scoring='f1', cv=cv
)
print("F1 CV média:", scores.mean())

probs_final = xgb_final.predict_proba(X_test)[:, 1]
preds_final = (probs_final >= 0.52).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_test, preds_final))


NameError: name 'y_train_resampled' is not defined

F1 médio no treino (CV): 0.92

- F1 na classe 1 no teste: 0.61

- Precision: 0.68

- Recall: 0.55

Interpretação:

- O modelo tá aprendendo bem no treino (dados balanceados).

- Mas no teste real, ainda sofre com recall (difícil prever quem sai da empresa).

- Gap entre treino e teste sugere que o comportamento da classe 1 no teste é diferente ou escassa.

### SHAP (SHapley Additive exPlanations)

In [ ]:
explainer = shap.Explainer(xgb_final)

shap_values = explainer(X_test)

shap.plots.beeswarm(shap_values)

NameError: name 'xgb_final' is not defined

In [ ]:
feature_names = X_train.columns.tolist()
print(len(feature_names), feature_names)

new_row = dict.fromkeys(feature_names, 0)

X_new = pd.DataFrame([{
    'Age': 32,
    'DailyRate': 800,
    'DistanceFromHome': 10,
    'Education': 3,
    'EmployeeNumber': 9999,
    'EnvironmentSatisfaction': 4,
    'Gender': 1,
    'HourlyRate': 60,
    'JobInvolvement': 3,
    'JobLevel': 2,
    'JobSatisfaction': 3,
    'MonthlyIncome': 4500,
    'MonthlyRate': 20000,
    'NumCompaniesWorked': 2,
    'PercentSalaryHike': 15,
    'PerformanceRating': 3,
    'RelationshipSatisfaction': 2,
    'StockOptionLevel': 0,
    'TotalWorkingYears': 8,
    'TrainingTimesLastYear': 2,
    'WorkLifeBalance': 3,
    'YearsAtCompany': 5,
    'YearsInCurrentRole': 2,
    'YearsSinceLastPromotion': 1,
    'YearsWithCurrManager': 3,
    'MonthlyIncome_log': np.log1p(4500),
    'TotalWorkingYears_log': np.log1p(8),
    'BusinessTravel_Travel_Frequently': 0,
    'BusinessTravel_Travel_Rarely': 1,
    'Department_Research & Development': 1,
    'Department_Sales': 0,
    'EducationField_Life Sciences': 0,
    'EducationField_Marketing': 0,
    'EducationField_Medical': 1,
    'EducationField_Other': 0,
    'EducationField_Technical Degree': 0,
    'JobRole_Human Resources': 0,
    'JobRole_Laboratory Technician': 0,
    'JobRole_Manager': 1,
    'JobRole_Manufacturing Director': 0,
    'JobRole_Research Director': 0,
    'JobRole_Research Scientist': 0,
    'JobRole_Sales Executive': 0,
    'JobRole_Sales Representative': 0,
    'MaritalStatus_Married': 1,
    'MaritalStatus_Single': 0,
    'OverTime_Yes': 0
}])

X_new = X_new[feature_names]

xgb_loaded = joblib.load('models/xgb_attrition_final.pkl')
threshold_loaded = joblib.load('models/threshold_optimizado.pkl')

probs_new = xgb_loaded.predict_proba(X_new)[:, 1]
pred_new  = (probs_new >= threshold_loaded).astype(int)

print(f"Prob. de saída: {probs_new[0]:.4f}")
print(f"Predição final: {pred_new[0]}  (1=sai, 0=fica)")


36 ['Age', 'BusinessTravel', 'DailyRate', 'Department', 'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction', 'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager', 'MonthlyIncome_log', 'TotalWorkingYears_log']


KeyError: "['BusinessTravel', 'Department', 'EducationField', 'EmployeeCount', 'JobRole', 'MaritalStatus', 'Over18', 'OverTime', 'StandardHours'] not in index"

Prob. de saída: 0.0116

(1=sai, 0=fica)
Predição final: 0 